https://scipy-cookbook.readthedocs.io/items/bundle_adjustment.html

Passpunkte

| 0  | 1  | 2  | 3  |
|----|----|----|----|
| 4  | 5  | 6  | 7  |
| 8  | 9 | 10 | 11 |
| 12 | 13 | 14 | 15 |

In [68]:
import time
import numpy as np
from scipy.optimize import least_squares
from scipy.sparse import lil_matrix

from PIL import Image
from glob import glob
import cv2
from cv2 import aruco
import numpy as np
from sympy import *
import math
from PIL import ExifTags
from xml.etree import ElementTree as ET
import utm
import math


In [69]:
passpunkte = np.array([
    [-0, -0, 0],
    [-1, -0, 0],
    [-2, -0, 0],
    [-3, -0, 0.1],
    [-0, -1, 0],
    [-1, -1, 0],
    [-2, -1, 0],
    [-3, -1, 0],
    [-0, -2, 0],
    [-1, -2, 0],
    [-2, -2, 0.1],
    [-3, -2, 0],
    [-0, -3, 0],
    [-1, -3, 0],
    [-2, -3, 0.1],
    [-3, -3, 0]
])


In [70]:
# kamera, bildnr, passpunkt, bildkoordinatenx, y
messungen = np.array([
    [0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0],
    [0, 0, 2, 2, 0],
    [0, 0, 4, 0, 1],
    [0, 0, 5, 1, 1],
    [0, 0, 6, 2, 1],
    [0, 0, 8, 0, 2],
    [0, 0, 9, 1, 2],
    [0, 0, 10, 2, 2],
    [0, 1, 1, 0, 0],
    [0, 1, 2, 1, 0],
    [0, 1, 3, 2, 0],
    [0, 1, 5, 0, 1],
    [0, 1, 6, 1, 1],
    [0, 1, 7, 2, 1],
    [0, 1, 9, 0, 2],
    [0, 1, 10, 1, 2],
    [0, 1, 11, 2, 2],
    [0, 2, 4, 0, 0],
    [0, 2, 5, 1, 0],
    [0, 2, 6, 2, 0],
    [0, 2, 8, 0, 1],
    [0, 2, 9, 1, 1],
    [0, 2, 10, 2, 1],
    [0, 2, 12, 0, 2],
    [0, 2, 13, 1, 2],
    [0, 2, 14, 2, 2],
    [0, 3, 5, 0, 0],
    [0, 3, 6, 1, 0],
    [0, 3, 7, 2, 0],
    [0, 3, 9, 0, 1],
    [0, 3, 10, 1, 1],
    [0, 3, 11, 2, 1],
    [0, 3, 13, 0, 2],
    [0, 3, 14, 1, 2],
    [0, 3, 15, 2, 2],
    [0, 4, 0, 0, 0],
    [0, 4, 1, 1, 0],
    [0, 4, 2, 2, 0],
    [0, 4, 3, 3, 0],
    [0, 4, 4, 0, 1],
    [0, 4, 5, 1, 1],
    [0, 4, 6, 2, 1],
    [0, 4, 7, 3, 1],
    [0, 4, 8, 0, 2],
    [0, 4, 9, 1, 2],
    [0, 4, 10, 2, 2],
    [0, 4, 11, 3, 2],
    [0, 4, 12, 0, 3],
    [0, 4, 13, 1, 3],
    [0, 4, 14, 2, 3],
    [0, 4, 15, 3, 3]
])

In [71]:
# x,y,z, r1, r2, r3, height, width
r1 = 0
r2 = 0
r3 = 0
bild = np.array([
    [1, 1, 1, r1, r2, r3, 2, 2],
    [2, 1, 1, r1, r2, r3, 2, 2],
    [1, 2, 1, r1, r2, r3, 2, 2],
    [2, 2, 1, r1, r2, r3, 2, 2],
    [1.5, 1.5, 1.5, r1, r2, r3, 3, 3]
])


In [72]:
# f, cx, cy
camera_params = np.array([[0.5, 0, 0]])

In [73]:
formeln = []


def r11(kappa, omega, phi):
    return cos(phi)*cos(kappa)-sin(phi)*sin(omega)*sin(kappa)


def r21(kappa, omega, phi):
    return sin(phi)*cos(kappa)+cos(phi)*sin(omega)*sin(kappa)


def r31(kappa, omega, phi):
    return -cos(omega)*sin(kappa)


def r12(kappa, omega, phi):
    return -sin(phi)*cos(omega)


def r22(kappa, omega, phi):
    return cos(phi)*cos(omega)


def r32(kappa, omega, phi):
    return sin(omega)


def r13(kappa, omega, phi):
    return cos(phi)*sin(kappa)+sin(phi)*sin(omega)*cos(kappa)


def r23(kappa, omega, phi):
    return sin(phi)*sin(kappa)-cos(phi)*sin(omega)*cos(kappa)


def r33(kappa, omega, phi):
    return cos(omega)*cos(kappa)


def kx(x, x0, y, y0, z, z0, kappa, omega, phi):
    return r11(kappa, omega, phi)*(x-x0)+r21(kappa, omega, phi)*(y-y0)+r31(kappa, omega, phi)*(z-z0)


def ky(x, x0, y, y0, z, z0, kappa, omega, phi):
    return r12(kappa, omega, phi)*(x-x0)+r22(kappa, omega, phi)*(y-y0)+r32(kappa, omega, phi)*(z-z0)


def nenner(x, x0, y, y0, z, z0, kappa, omega, phi):
    return r13(kappa, omega, phi)*(x-x0)+r23(kappa, omega, phi)*(y-y0)+r33(kappa, omega, phi)*(z-z0)


def x_kolinear(c, x, x0, y, y0, z, z0, kappa, omega, phi, deltax):
    return x0+c*kx(x, x0, y, y0, z, z0, kappa, omega, phi)/nenner(x, x0, y, y0, z, z0, kappa, omega, phi) + deltax


def y_kolinear(c, x, x0, y, y0, z, z0, kappa, omega, phi, deltay):
    return y0+c*ky(x, x0, y, y0, z, z0, kappa, omega, phi)/nenner(x, x0, y, y0, z, z0, kappa, omega, phi) + deltay


def kolinear(xy, c, x0, y0, z0, kappa, omega, phi, x, y, z, deltax, deltay):
    if xy == 0:
        return x_kolinear(c, x, x0, y, y0, z, z0, kappa, omega, phi, deltax)
    return y_kolinear(c, x, x0, y, y0, z, z0, kappa, omega, phi, deltay)

## Beginn Ausgleichung

In [88]:
l = np.array([[i[3]/bild[i[1]][6]-0.5, i[4]/bild[i[1]][7]-0.5]
              for i in messungen]).reshape(messungen[:, 3:].size)
l


array([-0.5, -0.5, 0.0, -0.5, 0.5, -0.5, -0.5, 0.0, 0.0, 0.0, 0.5, 0.0,
       -0.5, 0.5, 0.0, 0.5, 0.5, 0.5, -0.5, -0.5, 0.0, -0.5, 0.5, -0.5,
       -0.5, 0.0, 0.0, 0.0, 0.5, 0.0, -0.5, 0.5, 0.0, 0.5, 0.5, 0.5, -0.5,
       -0.5, 0.0, -0.5, 0.5, -0.5, -0.5, 0.0, 0.0, 0.0, 0.5, 0.0, -0.5,
       0.5, 0.0, 0.5, 0.5, 0.5, -0.5, -0.5, 0.0, -0.5, 0.5, -0.5, -0.5,
       0.0, 0.0, 0.0, 0.5, 0.0, -0.5, 0.5, 0.0, 0.5, 0.5, 0.5, -0.5, -0.5,
       -0.17, -0.5, 0.17, -0.5, 0.5, -0.5, -0.5, -0.17, -0.17, -0.17,
       0.17, -0.17, 0.5, -0.17, -0.5, 0.17, -0.17, 0.17, 0.17, 0.17, 0.5,
       0.17, -0.5, 0.5, -0.17, 0.5, 0.17, 0.5, 0.5, 0.5])

In [89]:
x0 = np.hstack((camera_params.flatten(), bild[:,:6].flatten(), passpunkte.flatten()))
x0

array([0.5, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 0.0,
       0.0, 0.0, 1.0, 2.0, 1.0, 0.0, 0.0, 0.0, 2.0, 2.0, 1.0, 0.0, 0.0,
       0.0, 1.5, 1.5, 1.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0,
       -2.0, 0.0, 0.0, -3.0, 0.0, 0.1, 0.0, -1.0, 0.0, -1.0, -1.0, 0.0,
       -2.0, -1.0, 0.0, -3.0, -1.0, 0.0, 0.0, -2.0, 0.0, -1.0, -2.0, 0.0,
       -2.0, -2.0, 0.1, -3.0, -2.0, 0.0, 0.0, -3.0, 0.0, -1.0, -3.0, 0.0,
       -2.0, -3.0, 0.1, -3.0, -3.0, 0.0])

In [90]:
sym = []

innereOrientierung = []
for camera in range(len(camera_params)):
    innereOrientierung0 = [
        Symbol('c'+str(camera)),
        Symbol('deltax'+str(camera)),
        Symbol('deltay'+str(camera))
    ]
    innereOrientierung.append(innereOrientierung0)
    sym.extend(innereOrientierung0)

bildOrientierung = []
for bildnr in range(len(bild)):
    bildO = []
    bildO.append(Symbol('bild'+str(bildnr)+'_x'))
    bildO.append(Symbol('bild'+str(bildnr)+'_y'))
    bildO.append(Symbol('bild'+str(bildnr)+'_z'))
    bildO.append(Symbol('bild'+str(bildnr)+'_kappa'))
    bildO.append(Symbol('bild'+str(bildnr)+'_omega'))
    bildO.append(Symbol('bild'+str(bildnr)+'_phi'))
    bildOrientierung.append(bildO)
    sym.extend(bildO)

neupunkte = []
for marker in range(len(passpunkte)):
    neup = []
    neup.append(Symbol('marker'+str(marker)+'x'))
    neup.append(Symbol('marker'+str(marker)+'y'))
    neup.append(Symbol('marker'+str(marker)+'z'))
    neupunkte.append(neup)
    sym.extend(neup)

print(sym)

[c0, deltax0, deltay0, bild0_x, bild0_y, bild0_z, bild0_kappa, bild0_omega, bild0_phi, bild1_x, bild1_y, bild1_z, bild1_kappa, bild1_omega, bild1_phi, bild2_x, bild2_y, bild2_z, bild2_kappa, bild2_omega, bild2_phi, bild3_x, bild3_y, bild3_z, bild3_kappa, bild3_omega, bild3_phi, bild4_x, bild4_y, bild4_z, bild4_kappa, bild4_omega, bild4_phi, marker0x, marker0y, marker0z, marker1x, marker1y, marker1z, marker2x, marker2y, marker2z, marker3x, marker3y, marker3z, marker4x, marker4y, marker4z, marker5x, marker5y, marker5z, marker6x, marker6y, marker6z, marker7x, marker7y, marker7z, marker8x, marker8y, marker8z, marker9x, marker9y, marker9z, marker10x, marker10y, marker10z, marker11x, marker11y, marker11z, marker12x, marker12y, marker12z, marker13x, marker13y, marker13z, marker14x, marker14y, marker14z, marker15x, marker15y, marker15z]


In [91]:
formeln = []
for bnr in range(len(messungen)):
    kamera, bildnr, punkt, x, y = messungen[bnr]
    bo = bildOrientierung[bildnr]
    neup = passpunkte[punkt]
    formeln.append(kolinear(
        0, innereOrientierung[kamera][0], bo[0], bo[1], bo[2], bo[3], bo[4], bo[5], neup[0], neup[1], neup[2], innereOrientierung[kamera][1], innereOrientierung[kamera][2]))
    formeln.append(kolinear(
        1, innereOrientierung[kamera][0], bo[0], bo[1], bo[2], bo[3], bo[4], bo[5], neup[0], neup[1], neup[2], innereOrientierung[kamera][1], innereOrientierung[kamera][2]))


for i in range(len(camera_params)):
    formeln.append(innereOrientierung[i][0])
    l = np.append(l, x0[i*3])

for bildnr in range(len(bildOrientierung)):
    for bildP in range(6):
        formeln.append(bildOrientierung[bildnr][bildP])
        l = np.append(l, bild[bildnr][bildP])

for passnr in range(len(passpunkte)):
    for passXYZ in range(3):
        formeln.append(neupunkte[passnr][passXYZ])
        l = np.append(l, passpunkte[passnr][passXYZ])


In [92]:
l

array([-0.5, -0.5, 0.0, -0.5, 0.5, -0.5, -0.5, 0.0, 0.0, 0.0, 0.5, 0.0,
       -0.5, 0.5, 0.0, 0.5, 0.5, 0.5, -0.5, -0.5, 0.0, -0.5, 0.5, -0.5,
       -0.5, 0.0, 0.0, 0.0, 0.5, 0.0, -0.5, 0.5, 0.0, 0.5, 0.5, 0.5, -0.5,
       -0.5, 0.0, -0.5, 0.5, -0.5, -0.5, 0.0, 0.0, 0.0, 0.5, 0.0, -0.5,
       0.5, 0.0, 0.5, 0.5, 0.5, -0.5, -0.5, 0.0, -0.5, 0.5, -0.5, -0.5,
       0.0, 0.0, 0.0, 0.5, 0.0, -0.5, 0.5, 0.0, 0.5, 0.5, 0.5, -0.5, -0.5,
       -0.17, -0.5, 0.17, -0.5, 0.5, -0.5, -0.5, -0.17, -0.17, -0.17,
       0.17, -0.17, 0.5, -0.17, -0.5, 0.17, -0.17, 0.17, 0.17, 0.17, 0.5,
       0.17, -0.5, 0.5, -0.17, 0.5, 0.17, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0,
       1.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 2.0, 1.0,
       0.0, 0.0, 0.0, 2.0, 2.0, 1.0, 0.0, 0.0, 0.0, 1.5, 1.5, 1.5, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, -2.0, 0.0, 0.0, -3.0, 0.0,
       0.1, 0.0, -1.0, 0.0, -1.0, -1.0, 0.0, -2.0, -1.0, 0.0, -3.0, -1.0,
       0.0, 0.0, -2.0, 0.0, -1.0, -2.0, 0.0, -2.0, -2

In [93]:
A_vorlage = []
for f in range(len(formeln)):
    zeile = []
    for s in range(len(sym)):
        formel = formeln[f].diff(sym[s])
        zeile.append(formel)
    A_vorlage.append(zeile)


In [94]:
P = np.eye(len(l))
print('P', P)


P [[1.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0 1.0 0.0 ... 0.0 0.0 0.0]
 [0.0 0.0 1.0 ... 0.0 0.0 0.0]
 ...
 [0.0 0.0 0.0 ... 1.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 1.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 1.0]]


In [96]:
werte = {}

for nr in range(len(sym)):
    werte[sym[nr]] = x0[nr]

#print(werte)

A = np.empty((len(formeln), len(sym)))
for f in range(len(formeln)):
    for s in range(len(sym)):
        A[f][s] = A_vorlage[f][s].evalf(subs=werte)


l0 = np.empty(len(l))
for i in range(len(l0)):
    l0[i] = formeln[i].evalf(subs=werte)

dl = l - l0

N = np.transpose(A)@P@A
h = np.transpose(A)@P@dl
Qx = np.linalg.inv(N)
dx = Qx@h

x0 = x0 + dx

print('x0', x0)

dx.sum()


x0 [0.52 -2.8 -2.8 1.4 1.4 0.78 0.027 -0.027 -0.00013 1.6 1.3 1.1 0.069 0.046
 -0.27 1.3 1.6 1.0 -0.018 -0.064 0.27 1.5 1.5 1.2 0.014 -0.031 -0.0029 1.7
 1.7 1.5 0.011 -0.0026 0.0015 0.0 0.0 0.0 -1.0 0.0 0.0 -2.0 0.0 0.0 -3.0
 0.0 0.1 0.0 -1.0 0.0 -1.0 -1.0 0.0 -2.0 -1.0 0.0 -3.0 -1.0 0.0 0.0 -2.0
 0.0 -1.0 -2.0 0.0 -2.0 -2.0 0.1 -3.0 -2.0 0.0 0.0 -3.0 0.0 -1.0 -3.0 0.0
 -2.0 -3.0 0.1 -3.0 -3.0 0.0]


-0.11739204279543491